In [ ]:
import requests as req
import pandas as pd
import sqlite3 as sql
import requests
from functools import reduce
import numpy as np
import os.path
from os import path

In [ ]:
sqlite_file = 'salaries_data.sql'

In [ ]:
db = sql.connect(":memory:")

In [ ]:
sql_file = open(sqlite_file)
sql_as_string = sql_file. read()
db. executescript(sql_as_string)

## Data  Manipulation

In [ ]:
salary_query= """
SELECT 
 firstname, lastname, department, salary, position
FROM Salaries
"""

In [ ]:
salaries = pd.read_sql(salary_query, db)

In [ ]:
salaries.head()

In [ ]:
def count_nonempty(d): 
    total = 0
    for k in d.keys():
        if (d.get(k) != None):
            total += 1
    print("the non empty total", total)

In [ ]:
headers = {
  'Accept': 'application/json'
}

In [ ]:
def fetch_profs(): 


    profs = dict()
    matched_profs = dict()

    for i in range(0, 10000, 1000):
        r = requests.get('https://api.planetterp.com/v1/professors?reviews=true', params={
          'limit': '1000',
          'offset': i,
        }, headers = headers)

        for e in r.json():
            profs[e.get("slug")] = e
            matched_profs[e.get("slug")] = None

    
    print(len(profs.values()))
    return (profs, matched_profs)

In [ ]:
def find_closest_match(name, matches):
    firstname = name.split(" ")[0]
    ret = [ m  for m in matches if m[0].find(firstname) >= 0 ]
    if(len(ret) > 0):
        return ret[0]
    return None
def calc_avg_review(revs):
    length = len(revs)
    if(length == 0):
        return np.nan
    total = 0
    for r in revs:
        total += r.get("rating")
    return total / length
        

In [ ]:
def get_department(dep, profs, matched_profs):
    matches = dict()

    for prof in profs.values():
        
        slug = prof.get("slug")
        name = prof.get("name")
        courses = prof.get("courses")
        avg_rating = calc_avg_review(prof.get("reviews"))
        
        if(matched_profs.get(slug) != None):
            continue

        lastname = name.split(" ")[-1]

        salary_query= f"""
            SELECT 
            firstname, 
            lastname,
            department,
            salary,
            position
            FROM Salaries
            WHERE lastname like "{lastname}"
            AND department = "{dep}"
        """
        rows = list(db.execute(salary_query))
        matches[lastname] = (len(rows), rows, name, slug, courses, avg_rating)
        

    for match in matches.keys():
        (length, v, name, slug, courses, avg_rating) = matches.get(match)
        if(length > 0):
           
            value = find_closest_match(name, v)
         
            if(value != None):
                
                matched_profs[slug] = (
                    value[0],
                    value[1],
                    value[2],
                    value[3],
                    value[4],
                    avg_rating,
                    slug,
                    name,
                    "-".join(courses)
                )


    return

In [ ]:
data = []

In [ ]:
def fetch_matched_salaries():
    if(path.isfile("matched_salaries.csv")):
        return pd.read_csv("matched_salaries.csv")
    
    
    (profs, matched_profs) = fetch_profs()
    
    arr = list(set(salaries["department"]))
    for dep in arr:   
        get_department(dep, profs, matched_profs)
        
    data = [ list(v)  for v in matched_profs.values() if v != None]
    matched_salaries = pd.DataFrame(data, columns=['firstname', 'lastname', 'department', 'salary', "position", "avg_rating", 'slug', "planetTerpName", "courses"])
    matched_salaries.to_csv(path_or_buf="matched_salaries.csv", index=False)
    return matched_salaries


In [ ]:
matched_salaries = fetch_matched_salaries()
matched_salaries.head()

In [ ]:
def fetch_reviewsData(_data):
    if(path.isfile("reviews.csv")):
        return pd.read_csv("reviews.csv")
        
    reviews_data = []
    for v in _data:

        r = requests.get('https://api.planetterp.com/v1/professor', params={
          'name': v,  'reviews': 'true'
        }, headers = headers).json()


        slug = r.get("slug")
        revs = r.get("reviews")
        if(revs == None):
            continue

        for rev in revs:  

            reviews_data.append([ 
                slug,
                rev.get("rating"),  
                rev.get("expected_grade"), 
                rev.get("course"),
                rev.get("review"),
            ])   
    reviews_df = pd.DataFrame(reviews_data, columns=['slug', 'rating', 'expected_grade', 'course', 'review'])
    reviews_df.to_csv(path_or_buf="reviews.csv", index=False)
    return  reviews_df



In [ ]:
reviews_df = fetch_reviewsData(list(matched_salaries["planetTerpName"]))
reviews_df.head()

In [ ]:
# reviews_df.to_csv(path_or_buf="reviews.csv", index=False)

In [ ]:
data

In [ ]:
# word vector classifications - gensim

# word cloud with reviews (bag of words)
    # words that describe popular profesors
    
# scatter plot of professor salaries vs ratings
# scatter plot of depart salaries vs ratings

# consider the title of the professors which may be a reason for increased salary


## EDA

In [ ]:
dept_avg = matched_salaries.groupby("department").mean().sort_values(by=["salary"], ascending=False)
dept_avg